In [1]:
import os
import glob

import numpy as np
import tensorflow as tf

from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
DATA_PATH = r'C:\Users\subrahm\Documents\breast-cancer-detection\Breast-Cancer-Detection\data\test_cnn_data'

In [3]:
num_images = len(list(glob.glob(os.path.join(DATA_PATH, '*/*/*.png'))))
"total images: {}".format(num_images)

'total images: 138'

In [4]:
train_dir = os.path.join(DATA_PATH,'train')
val_dir = os.path.join(DATA_PATH,'val')

In [ ]:
Image.open(list(glob.glob(train_dir+'/*/*.png'))[0])

In [6]:
CLASSES = np.array([item for item in os.listdir(train_dir)])
"classes: {}".format(CLASSES)

"classes: ['normal_tissue' 'tumor']"

In [7]:
print('Train:')
for cls in CLASSES:
    print(f"number of images belonging to {cls} class: {len(os.listdir(os.path.join(train_dir,cls)))}")
    
print('\nValidation:')
for cls in CLASSES:
    print(f"number of images belonging to {cls} class: {len(os.listdir(os.path.join(val_dir,cls)))}")

Train:
number of images belonging to normal_tissue class: 63
number of images belonging to tumor class: 63

Validation:
number of images belonging to normal_tissue class: 6
number of images belonging to tumor class: 6


In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300, 300),
        batch_size=10,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(300, 300),
        batch_size=10,
        class_mode='binary')

Found 126 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [ ]:
num_classes = 2

model = Sequential([
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.Conv2D(512, 3, padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu')
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs
)